In [ ]:
### Let's pull one of the SMART DS distribution feeder model 

from erad.utils.ditto_utils import download_smartds_data
download_smartds_data('P10U')


In [ ]:
# Let's export distribution assets

from erad.utils.opendss_utils import extract_export_opendss_model

extract_export_opendss_model('./smart_ds_downloads/P10U__2018__SFO__oedi-data-lake_opendss_no_loadshapes/Master.dss',
                             './opendss_assets')


In [ ]:
# Let's visualize the distribution network

import pandas as pd 
import plotly.graph_objects as go
import random

random.seed(3444)

buses_df = pd.read_csv('./opendss_assets/buses.csv')
bus_to_lat_mapping = dict(zip(buses_df['name'], buses_df['latitude']))
bus_to_lon_mapping = dict(zip(buses_df['name'], buses_df['longitude']))
lines_df = pd.read_csv('./opendss_assets/line_sections.csv', index_col='name')

lines_ = {}

plot_data = {'longitudes': [], 'latitudes': []}
for _, row in lines_df.iterrows():
    try:
        row = dict(row)
        line_longitudes = [bus_to_lon_mapping[row['source']], \
                                    bus_to_lon_mapping[row['target']], None]
        line_latitudes = [bus_to_lat_mapping[row['source']], \
                                    bus_to_lat_mapping[row['target']], None]
        plot_data['longitudes'] += line_longitudes
        plot_data['latitudes'] += line_latitudes
        
        lines_[_] = {
            "coordinates": [(line_latitudes[0] + line_latitudes[1])/2, \
                            (line_longitudes[0] + line_longitudes[1])/2],
            "heights_ft":  random.randint(-2, 9)*3.28084,
            "elevation_ft": random.randint(50, 400),
            "longitudes": line_longitudes,
            "latitudes": line_latitudes
        }
    except Exception as e:
        pass
data = go.Scattermapbox(
                mode="markers+lines",
                lon=plot_data["longitudes"],
                lat=plot_data["latitudes"],
                marker={"size": 0},
                line={"color": 'blue'},
            )
fig = go.Figure(data=data)
fig.update_mapboxes(
            {
                "accesstoken": None,
                "style": 'carto-darkmatter',
                "center": {
                   "lon": -122.81619708708354,
                    "lat": 38.6,
                },
                "zoom": 8,
            }
        )
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Let's run the flooding simulation

from erad.scenarios.flood_scenario import FlooadScenario
import json
from shapely import MultiPolygon, Polygon
import datetime

with open('flooding_polygon_naerm.geojson', "r") as fp:
                geojson = json.load(fp)

polygon_coords = [item['geometry']['coordinates'] for item in geojson['features'] \
                    if item['properties']['id'] == 1] 

coords = [[x[1], x[0]] for el in polygon_coords[0][0] for x in el]
           
scenario = FlooadScenario(
    MultiPolygon([Polygon(coords)]),
    None,
    datetime.datetime(2023,1,1,0,45,0),
    file_flow='./flows.csv',
    file_levels='./levels.csv',
    file_gaugues='./gauges.csv'
)

In [ ]:
# Get all line assets and their coordinates
import random
lines_dict = scenario.calculate_survival_probability({
    "distribution_overhead_lines": lines_
}, datetime.datetime(2023,1,1,0,45,0))


In [ ]:
# let's visualize assets affected by flooding

survived = [ subdict for li, subdict in lines_dict['distribution_overhead_lines'].items() \
                if random.random() <= subdict['survival_probability'] ]
not_survived = [ subdict for li, subdict in lines_dict['distribution_overhead_lines'].items() \
                if random.random() > subdict['survival_probability'] ]

survives_lines = {'longitudes': [], 'latitudes': []}
for li in survived:
    survives_lines['longitudes'] += li['longitudes']
    survives_lines['latitudes'] += li['latitudes']


not_survives_lines = {'longitudes': [], 'latitudes': []}
for li in not_survived:
    not_survives_lines['longitudes'] += li['longitudes']
    not_survives_lines['latitudes'] += li['latitudes']

data = [go.Scattermapbox(
                mode="markers+lines",
                lon=survives_lines["longitudes"],
                lat=survives_lines["latitudes"],
                marker={"size": 0},
                line={"color": 'blue'},
                name='survived'
            ),
        go.Scattermapbox(
                mode="markers+lines",
                lon=not_survives_lines["longitudes"],
                lat=not_survives_lines["latitudes"],
                marker={"size": 0},
                line={"color": 'red'},
                name='not survived'
        )
]
fig = go.Figure(data=data)
fig.update_mapboxes(
            {
                "accesstoken": None,
                "style": 'carto-darkmatter',
                "center": {
                   "lon": -122.81619708708354,
                    "lat": 38.6,
                },
                "zoom": 8,
            }
        )
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
    